In [3]:
import numpy as np
import pandas as pd
import math as m
import statsmodels.api as sm

# simple setting
\begin{aligned}
    &Y = \beta_0 + \beta_1*X + \beta_2U_2 + \epsilon\;\\
    &X = \beta_{0}' + \beta_1'U_1\\
    &Z = \beta_1''U_1 + \beta_2''U_2
\end{aligned}

$\beta_1 \sim \mathcal{N}(0.75, 0.2^2)$ and therefore, the average causal effect (ACE) is set to 0.75.

Further, $\beta_0 = 2, \beta_2 = 0.25, \beta_{0}'=0.1, \beta_1'=0.5, \beta_1''=0.2, \beta_2''=0.3$.

In [6]:
def _construct_data_simple(u1_params={'mean':2, 'sd':0.25}, 
                    u2_params={'mean':0, 'sd':1}, 
                    beta1_params={'mean':0.7, 'sd':0.2}, 
                    n=1000,
                    include_Z=False):
    """
    Constructs the dataset for the simple setting of the proposed unit test
    :param u1_params: dictionary of paramters for distribution of unobserved variable U1
    :param u2_params: dictionary of paramters for distribution of unobserved variable U2
    :param beta1_params: dictionary of paramters for distribution of the causal effect
    :param n: total sample size
    :param include_Z: flag for including or excliding the bad control
    :return: the features, and the outcome
    """
    U1 = np.random.normal(u1_params['mean'], u1_params['sd'], n)
    U2 = np.random.normal(u2_params['mean'], u2_params['sd'], n)
    beta1 = np.random.normal(beta1_params['mean'], beta1_params['sd'], n)

    X = 0.1 + 0.5*U1
    Z = 0.2*U1 + 0.3*U2
    Y = 2 + beta1*X + 0.25*U2

    if include_Z:
      return np.array([X, Z]).T, Y
    else:
      return np.expand_dims(X, axis=1), Y

In [7]:
def _regression_summary(X_data, Y_data):
    """
    Adds the intercept term and fits OLS
    :param X_data: the features
    :param Y_data: the outcome
    :return: the OLS regression table
    """
    X_data = sm.add_constant(X_data, prepend=True)
    model = sm.OLS(Y_data, X_data)
    result = model.fit()
    return result.summary()

In [29]:
X, Y  = _construct_data_simple(include_Z=False)
_regression_summary(X, Y)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.064
Model:                            OLS   Adj. R-squared:                  0.063
Method:                 Least Squares   F-statistic:                     67.95
Date:                Thu, 17 Nov 2022   Prob (F-statistic):           5.25e-16
Time:                        18:24:35   Log-Likelihood:                -384.62
No. Observations:                1000   AIC:                             773.2
Df Residuals:                     998   BIC:                             783.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.9430      0.100     19.345      0.000       1.746       2.140
x1             0.7489      0.091      8.243      0.000       0.571       0.927
==============================================================================
Omnibus:                        2.654   Durbin-Watson:                   2.121
Prob(Omnibus):                  0.265   Jarque-Bera (JB):                2.631
Skew:                           0.091   Prob(JB):                        0.268
Kurtosis:                       2.826   Cond. No.                         17.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

From the above table, we get $\beta_1 \approx 0.75$, with $CI \; [0.571, 0.927]$



In [35]:
X, Y = _construct_data_simple(include_Z=True)
_regression_summary(X, Y)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.626
Model:                            OLS   Adj. R-squared:                  0.626
Method:                 Least Squares   F-statistic:                     836.0
Date:                Thu, 17 Nov 2022   Prob (F-statistic):          6.58e-214
Time:                        18:25:54   Log-Likelihood:                 136.35
No. Observations:                1000   AIC:                            -266.7
Df Residuals:                     997   BIC:                            -252.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.0316      0.057     35.760      0.000       1.920       2.143
x1             0.3728      0.053      7.086      0.000       0.270       0.476
x2             0.8249      0.022     37.380      0.000       0.782       0.868
==============================================================================
Omnibus:                        4.270   Durbin-Watson:                   2.026
Prob(Omnibus):                  0.118   Jarque-Bera (JB):                4.247
Skew:                           0.120   Prob(JB):                        0.120
Kurtosis:                       3.211   Cond. No.                         17.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Now, when Z is included, $\beta_1 \approx 0.37$ with $CI \; [0.270, 0.476]$, which 
is a biased ACE estimate.

# generalized setting

\begin{aligned}
    Y &= \beta_0 + \beta_1*X + \beta_2U_2 + \mathbf{W}\theta + \epsilon + \delta\;\\
    X &= \beta_{0}' + \beta_1'U_1\\
    Z &= \beta_1''U_1 + \beta_2''U_2
\end{aligned}

Here the new terms are the neutral controls $W_1, ..., W_d$ contained in $\mathbf{W}$, and their coefficients $\theta \in \mathbb{R}^d$, along with the additional noise term $\delta$. The average causal effect is still set to 0.75.

In [8]:
def _make_theta(d=10, n=1000):
    """
    Makes the coefficients for the neutral controls
    :param d: the dimension of theta
    :param n: the total number of samples (used in computation of the thetas)
    :return: the theta vector
    """
    return np.array([10/(j*m.sqrt(n)) for j in range(1, d+1)]) 

In [113]:
def _construct_data_general(d=10, mean_eps = 0, r = 0.8, n = 1000, include_Z=False):
    """
    Constructs the data set for the generalized setting of the proposed unit test
    :param d: the number of neutral controls
    :param mean_eps: the mean of the additional noise term
    :param r: the coefficient of determination for the nutral controls and outcome
    :param n: total sample size
    :param include_Z: flag for including or excluding the bad control
    :return: the features, and the outcome
    """
    theta = _make_theta(d=d, n=n)
    var_delta = ((1-r)/r)*np.transpose(theta)@np.eye(d)@theta
    delta = np.random.normal(mean_eps, m.sqrt(var_delta), n)
    W = np.random.multivariate_normal(np.zeros(d), np.eye(d), n)

    X_data, Y_data = _construct_data_simple(n=n, include_Z=include_Z)

    Y_data = Y_data + W@theta + delta
    X_data = np.hstack([X_data, W])

    return X_data, Y_data

In [114]:
X_data, Y_data = _construct_data_general(include_Z=False)
_regression_summary(X_data, Y_data)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.539
Model:                            OLS   Adj. R-squared:                  0.534
Method:                 Least Squares   F-statistic:                     105.0
Date:                Thu, 17 Nov 2022   Prob (F-statistic):          1.11e-157
Time:                        22:56:48   Log-Likelihood:                -489.28
No. Observations:                1000   AIC:                             1003.
Df Residuals:                     988   BIC:                             1061.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.9264      0.112     17.185      0.000       1.706       2.146
x1             0.7631      0.101      7.550      0.000       0.565       0.961
x2             0.3246      0.012     26.852      0.000       0.301       0.348
x3             0.1595      0.013     12.066      0.000       0.134       0.185
x4             0.0890      0.013      7.044      0.000       0.064       0.114
x5             0.0653      0.012      5.259      0.000       0.041       0.090
x6             0.0764      0.013      5.749      0.000       0.050       0.102
x7             0.0310      0.013      2.478      0.013       0.006       0.056
x8             0.0687      0.012      5.545      0.000       0.044       0.093
x9             0.0312      0.012      2.513      0.012       0.007       0.056
x10            0.0340      0.013      2.687      0.007       0.009       0.059
x11            0.0222      0.012      1.801      0.072      -0.002       0.046
==============================================================================
Omnibus:                        3.258   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.196   Jarque-Bera (JB):                3.241
Skew:                           0.139   Prob(JB):                        0.198
Kurtosis:                       2.991   Cond. No.                         17.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

From the above table, we get $\beta_1 \approx 0.75$, with $CI \; [0.565, 0.961]$

In [116]:
X_data, Y_data = _construct_data_general(include_Z=True)
_regression_summary(X_data, Y_data)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.722
Model:                            OLS   Adj. R-squared:                  0.719
Method:                 Least Squares   F-statistic:                     213.8
Date:                Thu, 17 Nov 2022   Prob (F-statistic):          1.63e-264
Time:                        22:57:42   Log-Likelihood:                -239.62
No. Observations:                1000   AIC:                             505.2
Df Residuals:                     987   BIC:                             569.0
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.1035      0.086     24.598      0.000       1.936       2.271
x1             0.2946      0.078      3.796      0.000       0.142       0.447
x2             0.8490      0.032     26.336      0.000       0.786       0.912
x3             0.3190      0.010     32.359      0.000       0.300       0.338
x4             0.1572      0.010     15.967      0.000       0.138       0.176
x5             0.0972      0.010     10.108      0.000       0.078       0.116
x6             0.0798      0.010      8.111      0.000       0.060       0.099
x7             0.0643      0.009      6.918      0.000       0.046       0.083
x8             0.0603      0.010      6.154      0.000       0.041       0.079
x9             0.0495      0.010      5.033      0.000       0.030       0.069
x10            0.0503      0.010      4.818      0.000       0.030       0.071
x11            0.0260      0.010      2.654      0.008       0.007       0.045
x12            0.0286      0.010      2.923      0.004       0.009       0.048
==============================================================================
Omnibus:                        1.275   Durbin-Watson:                   2.068
Prob(Omnibus):                  0.528   Jarque-Bera (JB):                1.201
Skew:                           0.083   Prob(JB):                        0.549
Kurtosis:                       3.034   Cond. No.                         18.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Now, when Z is included, $\beta_1 \approx 0.29$ with $CI \; [0.142, 0.447]$ which is a biased ACE estimate.